In [1]:
# Trade data summary for ASX electricity contract

In [2]:
# Import libraries
import pandas as pd
import re

In [3]:
file_path = 'trades.csv'

In [4]:
contracts = ['Base', 'Cap']
states = ['NSW', 'QLD', 'VIC', 'SA']
expiries = ['Q1', 'Q2', 'Q3', 'Q4']
years = [2022, 2023, 2024, 2025, 2026]

In [5]:
# Define the mappings from the ASX electricity contract cheat sheet
contract_codes = {
    'B': 'Base', 'P': 'Peak', 'G': 'Cap', # $300 cap
    'H': 'Base_strip', 'D': 'Peak_strip', 'R': 'Cap_strip'
}

region_codes = {
    'N': 'NSW', 'Q': 'QLD', 'V': 'VIC', 'S': 'SA'
}

expiry_month_codes = {
    'F': 'JAN', 'G': 'FEB', 'H': 'MAR', 'J': 'APR', 
    'K': 'MAY', 'M': 'JUN', 'N': 'JUL', 'Q': 'AUG', 
    'U': 'SEP', 'V': 'OCT', 'X': 'NOV', 'Z': 'DEC'
}

quarter_codes = {
    'H': 'Q1', 'M': 'Q2', 'U': 'Q3', 'Z': 'Q4'
}

In [6]:
# Function to decode the product code to plain English
def decode(code):
    match = re.match(r'([A-Z])([A-Z])([A-Z])(\d{4})([A-Z]?)(\d{7})?', code)
    
    if match:
        contract = contract_codes.get(match.group(1), '')
        region = region_codes.get(match.group(2), '')
        expiry = quarter_codes.get(match.group(3), '')
        year = match.group(4)
        option_type = match.group(5)
        strike_price = match.group(6)
        
        if not contract:
            raise ValueError(f"Invalid contract code: {match.group(1)}")
        if not region:
            raise ValueError(f"Invalid region code: {match.group(2)}")
        if not expiry:
            raise ValueError(f"Invalid expiry code: {match.group(3)}")
        
        if option_type == "F":
            return f"{contract} {region} {expiry} {year}"
        elif option_type in ["C", "P"]:
            option = "Call option" if option_type == "C" else "Put option"
            strike_price_dollars = f"${int(strike_price) / 100:.2f}"
            return f"{contract} {region} {expiry} {year} {option} with a strike price of {strike_price_dollars}"
    
    return f"{code} - Invalid code"

In [7]:
# Function to encode plain English to product code
def encode(plain_english):
    try:
        parts = plain_english.split()
        
        if len(parts) == 4 or len(parts) == 8:
            contract = parts[0]
            region = parts[1]
            expiry = parts[2]
            year_code = parts[3]
            
            contract_code = next((k for k, v in contract_codes.items() if v == contract), None)
            region_code = next((k for k, v in region_codes.items() if v == region), None)
            expiry_code = next((k for k, v in quarter_codes.items() if v == expiry), None)
            
            if not contract_code:
                raise ValueError(f"Invalid contract code: {contract}")
            if not region_code:
                raise ValueError(f"Invalid region code: {region}")
            if not expiry_code:
                raise ValueError(f"Invalid expiry code: {expiry}")
            
            if len(parts) == 4:
                return f"{contract_code}{region_code}{expiry_code}{year_code}F"
            
            option_type = "C" if parts[4] == "Call" else "P"
            strike_price_cents = f"{int(float(parts[-1][1:]) * 100):07d}"
            return f"{contract_code}{region_code}{expiry_code}{year_code}{option_type}{strike_price_cents}"
        
        raise ValueError("Invalid input format.")
    
    except Exception as e:
        return f"Error: {str(e)}"

In [8]:
# Example usage of the above encode and decode functions
code = "BNH2022C0006000"
plain_english = "Cap QLD Q4 2025"

decoded = decode(code)
encoded = encode(plain_english)

print(f"Decoded: {decoded}")
print(f"Encoded: {encoded}")

Decoded: Base NSW Q1 2022 Call option with a strike price of $60.00
Encoded: GQZ2025F


In [9]:
def generate_trade_codes(contracts, states, expiries, years):
    trade_codes = []
    for contract in contracts:
        for state in states:
            for expiry in expiries:
                for year in years:
                    trade_code = f"{contract} {state} {expiry} {year}"
                    trade_codes.append(encode(trade_code))
    return trade_codes

trade_codes_of_interest = generate_trade_codes(contracts, states, expiries, years)
print(trade_codes_of_interest)

['BNH2022F', 'BNH2023F', 'BNH2024F', 'BNH2025F', 'BNH2026F', 'BNM2022F', 'BNM2023F', 'BNM2024F', 'BNM2025F', 'BNM2026F', 'BNU2022F', 'BNU2023F', 'BNU2024F', 'BNU2025F', 'BNU2026F', 'BNZ2022F', 'BNZ2023F', 'BNZ2024F', 'BNZ2025F', 'BNZ2026F', 'BQH2022F', 'BQH2023F', 'BQH2024F', 'BQH2025F', 'BQH2026F', 'BQM2022F', 'BQM2023F', 'BQM2024F', 'BQM2025F', 'BQM2026F', 'BQU2022F', 'BQU2023F', 'BQU2024F', 'BQU2025F', 'BQU2026F', 'BQZ2022F', 'BQZ2023F', 'BQZ2024F', 'BQZ2025F', 'BQZ2026F', 'BVH2022F', 'BVH2023F', 'BVH2024F', 'BVH2025F', 'BVH2026F', 'BVM2022F', 'BVM2023F', 'BVM2024F', 'BVM2025F', 'BVM2026F', 'BVU2022F', 'BVU2023F', 'BVU2024F', 'BVU2025F', 'BVU2026F', 'BVZ2022F', 'BVZ2023F', 'BVZ2024F', 'BVZ2025F', 'BVZ2026F', 'BSH2022F', 'BSH2023F', 'BSH2024F', 'BSH2025F', 'BSH2026F', 'BSM2022F', 'BSM2023F', 'BSM2024F', 'BSM2025F', 'BSM2026F', 'BSU2022F', 'BSU2023F', 'BSU2024F', 'BSU2025F', 'BSU2026F', 'BSZ2022F', 'BSZ2023F', 'BSZ2024F', 'BSZ2025F', 'BSZ2026F', 'GNH2022F', 'GNH2023F', 'GNH2024F', 'GN

In [10]:
def read_trade_data(file_path):
    # Skip the first line with the separator information
    df = pd.read_csv(file_path, skiprows=1)
    df['Date'] = pd.to_datetime(df['Date'])  # Convert Date column to datetime format
    return df

In [11]:
# Calculate the number of days a trade is away from the contract expiry
def get_expiry_date(contract_code):
    # Decode the contract code
    decoded_info = decode(contract_code)
    if "Invalid code" in decoded_info:
        return decoded_info
    
    # Extract expiry quarter and year
    parts = decoded_info.split()
    expiry_quarter = parts[2]
    expiry_year = parts[3]
    
    # Define the last day of each quarter
    last_days = {
        'Q1': '31/03',
        'Q2': '30/06',
        'Q3': '30/09',
        'Q4': '31/12'
    }
    
    # Get the last day for the given quarter
    last_day = last_days.get(expiry_quarter)
    
    if last_day:
        # Combine with the given year and convert to datetime object
        return pd.to_datetime(f"{last_day}/{expiry_year}", format="%d/%m/%Y")
    else:
        return "Invalid quarter"

In [12]:
def process_trade_data(df, trade_code):
    # Filter the dataframe by the specified trade code
    filtered_df = df[df['Code'] == trade_code]

    # Group by date and calculate the required sums
    grouped_df = filtered_df.groupby('Date').agg({
        'Cleared Volume': 'sum',
        'Face Value': 'sum',
        'Volume x MWh': 'sum',
        'Code': 'first',
        'Product': 'first',
        'Location': 'first',
        'Trade Type': 'first',
        'Period': 'first',
        'Year': 'first'
    }).reset_index()

    # Rename the columns
    grouped_df.rename(columns={
        'Cleared Volume': 'Daily contracts traded',
        'Face Value': 'Daily dollars traded',
        'Volume x MWh': 'Daily MWh traded'
    }, inplace=True)

    # Calculate the daily volume weighted average price in $/MWh
    grouped_df['Daily average price'] = grouped_df['Daily dollars traded'] / grouped_df['Daily MWh traded']

    # Sort values by Date before calculating cumulative sums
    grouped_df.sort_values(by=['Date'], inplace=True)

    # Calculate the cumulative sums
    grouped_df['Cumulative MWh traded'] = grouped_df['Daily MWh traded'].cumsum()
    grouped_df['Cumulative dollars traded'] = grouped_df['Daily dollars traded'].cumsum()

    # Calculate the cumulative trade volume-weighted average price in $/MWh
    grouped_df['Cumulative average price'] = grouped_df['Cumulative dollars traded'] / grouped_df['Cumulative MWh traded']

    # Generate a date range covering all dates from the min to max date in the data
    all_dates = pd.date_range(start=grouped_df['Date'].min(), end=grouped_df['Date'].max())

    # Forward fill missing values until the last day of trade
    grouped_df = grouped_df.set_index('Date').reindex(all_dates).fillna(method='ffill')

    # Reset the index to make the date column a regular column again
    grouped_df = grouped_df.reset_index().rename(columns={'index': 'Date'})

    # Calculate the number of days of the trade from contract expiry
    expiry_date = get_expiry_date(trade_code)
    grouped_df['Days from expiry'] = grouped_df['Date'].apply(lambda x: (expiry_date - x).days)

    return grouped_df


In [13]:
df = read_trade_data(file_path)

In [14]:
result_df = process_trade_data(df, trade_codes_of_interest[0])
print(result_df)

           Date  Daily contracts traded  Daily dollars traded  \
0    2019-05-07                    10.0             1778760.0   
1    2019-05-08                    10.0             1778760.0   
2    2019-05-09                    10.0             1778760.0   
3    2019-05-10                    10.0             1778760.0   
4    2019-05-11                    10.0             1778760.0   
...         ...                     ...                   ...   
1052 2022-03-24                     3.0              555120.0   
1053 2022-03-25                     3.0              555120.0   
1054 2022-03-26                     3.0              555120.0   
1055 2022-03-27                     3.0              555120.0   
1056 2022-03-28                     2.0              378000.0   

      Daily MWh traded      Code Product Location  Trade Type Period    Year  \
0              21600.0  BNH2022F  Future      NSW  Strip Legs     Q1  2022.0   
1              21600.0  BNH2022F  Future      NSW  Strip Le

In [15]:
# Initialize an empty DataFrame to store the final results
final_df = pd.DataFrame()

# Loop through each trade code in the list of interest
for trade_code in trade_codes_of_interest:
    # Call the process_trade_data function and get the result for the current trade code
    result_df = process_trade_data(df, trade_code)
    
    # Append the result_df to the final_df
    final_df = pd.concat([final_df, result_df], ignore_index=True)

# Now final_df contains the appended results for all trade codes


In [16]:
final_df.to_csv('new_summary.csv')